#[1] Mount Google Drive

In [ ]:
from google.colab import drive
import os
import sys
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define Path
path_records = '/content/drive/MyDrive/Github/Article/records'
# Set Path
path_wd = '/content/drive/MyDrive/Github/Article'
path_font = '/content/drive/MyDrive/Github/Article/fonts/STHUPO.TTF'
path_img = '/content/drive/MyDrive/Github/Article/img'
path_reg = '/content/drive/MyDrive/Github/Content/sources/ChinaFilm'

os.chdir(path_wd)

# Install requirements and restart

In [ ]:
%%capture
!pip install pandas==1.3.2
!pip install transformers
# Install Requirements
!pip install -r /content/drive/MyDrive/Github/Content/sources/ChinaFilm/requirements.txt
!apt update
#!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# Import Libraries

In [ ]:
# Import External Libs
import pandas as pd
import numpy as np
import datetime as dt
import re
import random

from IPython.display import display, HTML
import torch
from transformers import AdamW, BartForConditionalGeneration

from selenium import webdriver

import jieba.analyse
tfidf = jieba.analyse.extract_tags
textrank = jieba.analyse.textrank

import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import matplotlib.font_manager as fm

path_fonts = '/content/drive/MyDrive/Github/Article/fonts'
fontprop = fm.FontProperties(fname=path_fonts, size= 15)

font_dirs = [path_fonts, ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)
#font_list = fm.createFontList(font_files)
for font in font_files:
  fm.fontManager.addfont(font)

plt.rcParams['figure.figsize'] = [15, 9]
mp.rcParams['font.family'] = ['Microsoft YaHei']

%matplotlib inline

In [ ]:
# Load Libraries Local
path_Content = '/content/drive/MyDrive/Github/Content'
os.chdir(path_Content)

#%load_ext autoreload
%reload_ext autoreload


from sources.ChinaFilm.Record_Registration import Registration
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Initialize Registration Class
driver = webdriver.Chrome('chromedriver',options=chrome_options)
dy_reg = Registration(driver)

%reload_ext autoreload
os.chdir(path_wd)
from generate import utils

/content/drive/MyDrive/Github/Article/generate/utils.py:21: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.
  font_list = fm.createFontList(font_files)
Installing paddle-tiny, please wait a minute......
Paddle enabled successfully......


# [3] Load Release and Registration Records

## ChinaFilm 电影局

In [5]:
# Update Release Records
path_reg = '/content/drive/MyDrive/Github/Content/sources/ChinaFilm'
os.chdir(path_reg)
#!python Update_Release.py

In [6]:
# Update Reg Records
path_reg = '/content/drive/MyDrive/Github/Content/sources/ChinaFilm'
os.chdir(path_reg)
#!python Update_Registration.py

## Maoyan 猫眼

In [8]:
# Update Maoyan Releases - Run only if necessary
path_maoyan = '/content/drive/MyDrive/Github/Content/sources/maoyan'
os.chdir(path_maoyan)
!python search_curr_releases.py

search result saved to: searches/search__20211025_2253.json  with 22 records
file saved to: records_maoyan.json with a total of  22  records.


In [9]:
path_maoyan_releases = path_maoyan + '/records/' + 'records_maoyan.json'
df_maoyan = pd.read_json(path_maoyan_releases)
df_maoyan = df_maoyan.drop_duplicates('fid')

In [ ]:
df_maoyan

In [175]:
# Import Releases
path_releases = '/content/drive/MyDrive/Github/Content/sources/ChinaFilm/records/contents_of_releases.json'
df_releases = pd.read_json(path_releases)

# Import Reg
path_regs = '/content/drive/MyDrive/Github/Content/sources/ChinaFilm/records/contents_of_registrations.json'
df_reg = pd.read_json(path_regs, orient='record')
df_reg['匹配片名'] = df_reg['片名']

# Get Latest Issue
str_date_curr_release = df_releases.sort_values(by='公示日期', ascending=False).iloc[0]['公示日期']
dt_curr_release = dt.datetime.strptime(str_date_curr_release, '%Y-%m-%d')
df_curr_releases = df_releases[df_releases['公示日期']==str_date_curr_release]
df_curr_releases = df_curr_releases[['电影类别', '编码', '公映证号', '片名', '第一出品单位']].copy()

In [ ]:
# Merge Maoyan info into df_curr_releases
df_curr_releases = df_curr_releases.merge(df_maoyan, on='片名', how='left')
df_curr_releases.info()

In [12]:
# Export current release to csv for Power Automate processing
df_curr_releases.to_csv(
    path_records + '/df_curr_releases_{}.csv'.format(str_date_curr_release),
    encoding='utf-8-sig'
)

In [242]:
str_date_curr_release

'2021-10-14'

In [14]:
df_curr_releases.columns

Index(['电影类别', '编码', '公映证号', '片名', '第一出品单位', 'fid', '主演', '上映日期', '想看'], dtype='object')

In [15]:
df_reg.columns

Index(['备案立项号', '片名', '备案单位', '编剧', '备案结果', '备案地', '梗概', '公示日期', '公示批次名称',
       '备案详细页链接', '公示批次链接', '电影类别', '匹配片名'],
      dtype='object')

#[4] Interactively Find matches

In [16]:
# Load Libraries Local
path_Baike = '/content/drive/MyDrive/Github/Content/sources/baike'
os.chdir(path_Baike)

#%load_ext autoreload
%reload_ext autoreload
from Baike import Baike
baike = Baike()

In [ ]:
# Check for unmatched records and perform mannual matches
df_curr_releases['匹配片名'] = df_curr_releases['片名']
df_matched = df_reg[df_reg['匹配片名'].isin(df_curr_releases['片名'].to_numpy())]
idx_no_match = df_curr_releases.loc[~df_curr_releases['片名'].isin(df_matched['片名'].to_numpy())].index
df_curr_releases.loc[df_curr_releases.index.isin(idx_no_match), '匹配片名'] = None
df_curr_releases.loc[df_curr_releases['匹配片名'].isna(), :]

In [166]:
df_curr_releases.loc[df_curr_releases['匹配片名'].isna(), '片名'].to_numpy()

array(['迷途重生', '北方无故人', '城南老街新戏', '疯狂的熊孩子2', '伴我远行', '峰爆', '烈马争锋上海滩',
       '飞雪迎春', '嫌疑人之不可撤销', '有我们无数的好兄弟', '守望青春', '南游记之闹三界', '神龟岛',
       '花儿为什么这样红', '杨得操家的幸福', '情满天山', '遇见你之后'], dtype=object)

In [179]:
# assign manually found titles to df_curr_releases
found = [None, '风车', '老街新戏', '疯狂熊孩子2', '我的三次旅行', 
         '无限深度', '鲜衣怒马','多重身份', '嫌疑人3', '勇士的歌谣', 
         '高校辅导员', '南游记之三眼神将', '仙人岛', None, 
         '杨德操家的幸福', '我的新疆亲戚', '罗马香情']
df_curr_releases.loc[df_curr_releases['匹配片名'].isna(), '匹配片名'] = found

In [ ]:
df_reg.loc[df_reg['备案单位'].fillna('').str.contains('庆阳汇恒盛视影视文化传媒有限公司'), 
           ['片名', '备案立项号', '备案单位', '编剧', '梗概', '公示日期']]#.loc[6925, '梗概']

In [ ]:
df_reg.loc[df_reg['梗概'].fillna('').str.contains('迷途')]#.loc[4268,'梗概']

In [ ]:
df_reg.loc[df_reg['编剧'].fillna('').str.contains('祖辛'), 
           ['片名', '备案单位', '编剧', '梗概', '备案立项号']] 

,片名,备案单位,编剧,梗概,备案立项号
7091,罗马香情,北京无敌影业有限公司、奥瑞萨影视制作有限公司、铎飞影业有限公司,阿尔贝托·西蒙尼、余思、侯祖辛,偶像歌手彭柏杨跟随剧组到罗马拍摄真人秀，意外走失后与定居在当地的年轻女孩何曼曼相遇，两人一起...,影合立字（2019）第37号


In [ ]:
data = baike.search_for_content('守护码头')
baike.parse_content(data)

In [ ]:
pd.read_html(data)

In [ ]:
df_curr_releases

In [137]:
df_matched.columns

Index(['备案立项号', '片名', '备案单位', '编剧', '备案结果', '备案地', '梗概', '公示日期', '公示批次名称',
       '备案详细页链接', '公示批次链接', '电影类别', '匹配片名'],
      dtype='object')

In [138]:
df_reg.columns

Index(['备案立项号', '片名', '备案单位', '编剧', '备案结果', '备案地', '梗概', '公示日期', '公示批次名称',
       '备案详细页链接', '公示批次链接', '电影类别', '匹配片名'],
      dtype='object')

In [180]:
# Merge Manually Identified name
df_matched = df_reg.loc[df_reg['匹配片名'].isin(df_curr_releases['匹配片名'].to_numpy()),
                        ['匹配片名', '编剧', '梗概', '公示日期', 
                         '备案单位', '备案结果']
                       ]
df_matched.rename(columns={'公示日期':'备案公示日期'}, inplace=True)
df = df_curr_releases.merge(df_matched, on='匹配片名', how='left')
df['备案公示日期'] = df['备案公示日期'].apply(
    lambda x: pd.to_datetime(x, infer_datetime_format=True))
df['备案至今'] = dt_curr_release - df['备案公示日期']

In [212]:
df.shape

(55, 23)

In [ ]:
df_matched

# [5] Run Models for prediction

## Predict Genre

In [184]:
# Define Labels
label2id_genre= {'传奇': 0, '传记': 1, '其它': 2, '军旅': 3, '农村': 4, '宫廷': 5,  
                  '武打': 6, '涉案': 7, '神话': 8, '科幻': 9, '都市': 10, '青少': 11,
                  '革命': 12}
id2label_genre = {}
for k, v in label2id_genre.items(): id2label_genre[v] = k

In [185]:
%%capture
# Genre Classifier
N_LABELS = len(id2label_genre)
ID2LABEL = id2label_genre
LABEL2ID = label2id_genre

from transformers import BertTokenizer, BertForSequenceClassification
import torch
############
model_name = '/content/drive/MyDrive/Github/Content/tools/models/chinese-bert-wwm-classification-NRTAgenre'
############
if torch.cuda.device_count() > 0:
  device = 'cuda:' + str(torch.cuda.current_device())
else:
  device = 'cpu'

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(
  model_name,
  num_labels=N_LABELS,
  problem_type="multi_label_classification",
  id2label=ID2LABEL,
  label2id=LABEL2ID,  
).to(device)
model.eval() # set model to eval mode for faster prediction

In [186]:
batch_size = 8
i = 0
ls = df['梗概'].fillna('').tolist()
softmax = torch.nn.Softmax(dim=-1)
L = df.shape[0]
test_predictions = []

while i < L:
  batch_test = tokenizer(ls[i:i+batch_size],
                           padding=True,
                           max_length=512, 
                           truncation=True, 
                           return_tensors='pt')
  batch_test.to(device)
  batch_outputs = model(**batch_test)
  batch_logtis = batch_outputs.logits
  batch_softmax = softmax(batch_logtis)
  batch_results = torch.argmax(batch_softmax, dim=1).cpu().numpy()
  test_predictions.extend(list(batch_results))
  i += batch_size

df['类型'] = test_predictions
df['类型'] = df['类型'].apply(lambda x: id2label_genre[x])

## Predict Time

In [187]:
# Define Labels
label2id_time = {'古代': 0, '当代': 1, '现代': 2, '近代': 3}
id2label_time = {}
for k, v in label2id_time.items(): id2label_time[v] = k

In [188]:
%%capture
# Time Classifier

N_LABELS = len(id2label_time)
ID2LABEL = id2label_time
LABEL2ID = label2id_time

from transformers import BertTokenizer, BertForSequenceClassification
import torch
############
model_name = '/content/drive/MyDrive/Github/Content/tools/models/chinese-bert-wwm-classification-NRTAtime'
############
if torch.cuda.device_count() > 0:
  device = 'cuda:' + str(torch.cuda.current_device())
else:
  device = 'cpu'

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(
  model_name,
  num_labels=N_LABELS,
  problem_type="multi_label_classification",
  id2label=ID2LABEL,
  label2id=LABEL2ID,  
).to(device)
model.eval() # set model to eval mode for faster prediction

In [189]:
batch_size = 8
i = 0
ls = df['梗概'].fillna('').tolist()
softmax = torch.nn.Softmax(dim=-1)
L = df.shape[0]
test_predictions = []

while i < L:
  batch_test = tokenizer(ls[i:i+batch_size],
                           padding=True,
                           max_length=512, 
                           truncation=True, 
                           return_tensors='pt')
  batch_test.to(device)
  batch_outputs = model(**batch_test)
  batch_logtis = batch_outputs.logits
  batch_softmax = softmax(batch_logtis)
  batch_results = torch.argmax(batch_softmax, dim=1).cpu().numpy()
  test_predictions.extend(list(batch_results))
  i += batch_size

df['年代'] = test_predictions
df['年代'] = df['年代'].apply(lambda x: id2label_time[x])

## Predict Title

In [190]:
%%capture
from transformers import BertTokenizer, BartForConditionalGeneration

# assign device
if torch.cuda.device_count() > 0:
  device = 'cuda:' + str(torch.cuda.current_device())
else:
  device = 'cpu'

# Instantiate tokenizer and model
checkpoint = "/content/drive/MyDrive/Github/Content/tools/models/PredTitle-10000"

  
tokenizer = BertTokenizer.from_pretrained(checkpoint)
model = BartForConditionalGeneration.from_pretrained(checkpoint,
                                            gradient_checkpointing=True, 
                                            use_cache=False)
model.to(device)
model.eval()

batch_size = 8
i = 0
ls = df['梗概'].fillna('').tolist()
L = df['梗概'].shape[0]
test_predictions = []

while i < L:
  inputs = tokenizer(ls[i:i+batch_size],
                           padding=True,
                           max_length=512, 
                           truncation=True, 
                           return_tensors='pt')
  inputs.to(device)
  summary_ids = model.generate(input_ids=inputs['input_ids'],
                             num_beams=4,
                             min_length=0,
                             max_length=32
                             )
  
  ret = [tokenizer.decode(g, 
                         skip_specical_tokens=True, 
                         clean_up_tokenization_spaces=True) for g in summary_ids]
  test_predictions.extend(ret)
  i += batch_size

df['预测片名'] = test_predictions

def remove_specials(x):
  x = re.sub(' ', '', x)
  x = re.sub('\[CLS\]', '', x)
  x = re.sub('\[PAD\]', '', x)
  x = re.sub('\[SEP\]', '', x)
  return x

df['预测片名'] = df['预测片名'].apply(remove_specials)

## Extract top keywords

In [191]:
import jieba.analyse
tfidf = jieba.analyse.extract_tags
textrank = jieba.analyse.textrank

df['kw'] = df['梗概'].fillna('').apply(textrank, topK=10)

## Identify Main Characters

In [192]:
import jieba
import jieba.posseg as pseg
jieba.enable_paddle() #启动paddle模式。 0.40版之后开始支持，早期版本不支持

Paddle enabled successfully......


In [193]:
# 识别主要角色
def find_PER(x: pd.Series):
  words = pseg.cut(x, use_paddle=True)
  ls_PER = []
  for word, flag in words:
    if flag == 'PER':
      word = word.strip('[\s,，。、“”？；]')
      ls_PER.append(word)
  ls_PER = list(set(ls_PER))

  ls_PER1 = ls_PER.copy()
  ls_PER = []
  while ls_PER1:
    PER =  ls_PER1.pop(0)
    ls_tmp = re.split('[\s,，。、“”？；]', PER)
    ls_PER.extend(ls_tmp)

  ls_PER = list(set(ls_PER)) 
  ls_PER = [PER for PER in ls_PER if len(PER) > 1]
  ls_PER_sorted = sorted(ls_PER, reverse=False, key=lambda x: len(x))

  ans = []
  while ls_PER_sorted:
    PER = ls_PER_sorted.pop(0)
    i = 0
    while i < len(ls_PER_sorted):
      if re.search(PER, ls_PER_sorted[i]):
        ls_PER_sorted.pop(i)
        ans.append(PER)
      i += 1
    if PER not in ans:
      ans.append(PER)

  return ans

In [194]:
df['主要角色'] = df['梗概'].fillna('').apply(find_PER)

## adjust genre

In [195]:
df['修正类型'] = df['类型'].copy()
df['类型'].to_numpy()

array(['革命', '其它', '都市', '都市', '农村', '传奇', '都市', '都市', '都市', '传奇', '青少',
       '科幻', '都市', '都市', '涉案', '都市', '青少', '农村', '革命', '都市', '都市', '其它',
       '都市', '都市', '其它', '都市', '其它', '传奇', '革命', '涉案', '其它', '革命', '传奇',
       '传奇', '革命', '其它', '都市', '神话', '传奇', '革命', '其它', '神话', '其它', '其它',
       '都市', '都市', '农村', '都市', '军旅', '农村', '农村', '都市', '都市', '都市', '都市'],
      dtype=object)

In [245]:
adjusted = ['革命', '科幻', '都市', '都市', '农村', '传奇', '都市', '都市', '都市', '传奇', '青少',
       '科幻', '都市', '都市', '涉案', '都市', '青少', '农村', '革命', '都市', '都市', '革命',
       '都市', '都市', '其它', '都市', '其它', '传奇', '革命', '涉案', '其它', '革命', '传奇',
       '传奇', '革命', '其它', '都市', '神话', '传奇', '革命', '其它', '神话', '其它', '其它',
       '其它', '都市', '农村', '都市', '军旅', '农村', '农村', '都市', '都市', '都市', '都市']

df['修正类型'] = adjusted

In [ ]:
df[['片名', '类型', '修正类型', '梗概']]

## Save Records Curr Issue

In [246]:
# Save records for current release issue
df.to_pickle(path_records + '/df_releases_{}.pkl'.format(str_date_curr_release))

In [ ]:
#df = pd.read_pickle(path_records + '/df_releases_{}.pkl'.format(str_date_curr_release))

In [ ]:
df.info()

## View Table


In [247]:
# Formulat view table
tmp = df[['备案单位','第一出品单位']].fillna('').apply(
    lambda x: re.search(x['第一出品单位'], x['备案单位']), axis=1).to_frame()
idx_co_change = tmp[tmp[0].isna()].index
df['公司变更'] = 0
df.loc[idx_co_change, '公司变更'] = 1
df_view = df[['片名', '匹配片名','梗概', '备案至今', '备案结果', 
              '第一出品单位', '备案单位', '公司变更', '编剧', 
              '类型', '年代', '上映日期', '主演', '预测片名', 'kw',
              '主要角色', '修正类型'
              ]
            ].dropna(subset=['梗概'])
df_view.loc[df_view['片名'] == df_view['匹配片名'], '匹配片名'] = ''
df_view.rename(columns={'匹配片名':'备案名'}, inplace=True)
df_view.loc[df_view['公司变更'] == 0, '备案单位'] = ''
df_view.rename(columns={'备案单位':'原备案单位'}, inplace=True)
df_view.loc[df_view['备案结果'].str.contains('修改'),'备案结果'] = '是'
df_view.loc[df_view['备案结果'].str.contains('同意'),'备案结果'] = ''
df_view.rename(columns={'备案结果':'是否修改'}, inplace=True)
df_view['备案至今'] = round((df_view['备案至今'].dt.days + 60) / 365, 1)
df_view.sort_values(by='备案至今', ascending=False, inplace=True)
df_view['备案至今'] = df_view['备案至今'].apply(lambda x: '> ' + str(x) + ' 年')
df_view.drop('公司变更', axis=1, inplace=True)
df_view.loc[df_view['上映日期'].str.contains('今天').fillna(False),
            '上映日期'] = '上映中'

In [248]:
df_genre_sorted = df_view.groupby('类型')['片名'].count().rename(
    '数量').reset_index().sort_values('数量', ascending=False)
df_genre_sorted

,类型,数量
7,都市,19
1,其它,9
0,传奇,6
9,革命,6
3,农村,5
4,涉案,2
5,神话,2
8,青少,2
2,军旅,1
6,科幻,1


# [6] Generate WordCloud

In [200]:
from IPython.display import Image as Img
from PIL import Image as pil

path_posters = '/content/drive/MyDrive/Github/Article/img/posters'
path_icon = '/content/drive/MyDrive/Github/Article/img/genre_icon'

#df_label2image.to_json(path_records + '/df_label2image.json')
df_label2image = pd.read_json(path_wd + '/records/df_label2image.json')

In [201]:
id2label_time = {0: '古代', 1: '当代', 2: '现代', 3: '近代'}

id2label_genre = {0: '传奇', 1: '传记', 2: '其它', 3: '军旅', 4: '农村',
                  5: '宫廷', 6: '武打',  7: '涉案',   8: '神话',  9: '科幻',
                  10: '都市',  11: '青少',  12: '革命'
                 }
d_img_labels = {}
for t in id2label_time.values():
  for g in id2label_genre.values():
    d_img_labels[(t, g)] = None
d_img_labels.keys()

dict_keys([('古代', '传奇'), ('古代', '传记'), ('古代', '其它'), ('古代', '军旅'), ('古代', '农村'), ('古代', '宫廷'), ('古代', '武打'), ('古代', '涉案'), ('古代', '神话'), ('古代', '科幻'), ('古代', '都市'), ('古代', '青少'), ('古代', '革命'), ('当代', '传奇'), ('当代', '传记'), ('当代', '其它'), ('当代', '军旅'), ('当代', '农村'), ('当代', '宫廷'), ('当代', '武打'), ('当代', '涉案'), ('当代', '神话'), ('当代', '科幻'), ('当代', '都市'), ('当代', '青少'), ('当代', '革命'), ('现代', '传奇'), ('现代', '传记'), ('现代', '其它'), ('现代', '军旅'), ('现代', '农村'), ('现代', '宫廷'), ('现代', '武打'), ('现代', '涉案'), ('现代', '神话'), ('现代', '科幻'), ('现代', '都市'), ('现代', '青少'), ('现代', '革命'), ('近代', '传奇'), ('近代', '传记'), ('近代', '其它'), ('近代', '军旅'), ('近代', '农村'), ('近代', '宫廷'), ('近代', '武打'), ('近代', '涉案'), ('近代', '神话'), ('近代', '科幻'), ('近代', '都市'), ('近代', '青少'), ('近代', '革命')])

In [ ]:
from IPython.display import Image as Img
from PIL import Image as pil

path_posters = '/content/drive/MyDrive/Github/Article/img/posters'
path_icon = '/content/drive/MyDrive/Github/Article/img/genre_icon'

#df_label2image.to_json(path_records + '/df_label2image.json')
df_label2image = pd.read_json(path_records + '/df_label2image.json')

for i, row in df_label2image.tail(13).iterrows():
  image = pil.open(path_icon + '/' + row['fn'])
  #image.thumbnail([300, sys.maxsize], pil.ANTIALIAS)
  #image.save(path_posters + '/' + row['img_poster'], format='png')
  display(Img(path_icon + '/' + row['fn'], width=300))

In [ ]:
df.columns

Index(['电影类别', '编码', '公映证号', '片名', '第一出品单位', '备案名', '备案立项号', '备案单位', '编剧',
       '备案结果', '备案地', '梗概', '备案公示日期', '备案至今', 'time', 'genre', '类型', '年代',
       '预测片名', 'kw', '主要角色', '公司变更'],
      dtype='object')

In [ ]:
for i, f in df.drop_duplicates('片名'):
  ls = [f['片名']]*10
  ls += [f['预测片名']]*6
  ls += [f['主要角色']]*3
  if f['']

# [7] Generate Article

##[7.1] T1 标题：影片的前世今生

In [249]:
# Title
T1 = '~{}月新片的前世今生~{}'.format(dt_curr_release.month, dt_curr_release.year)
T1

'~10月新片的前世今生~2021'

##[7.2] P1 2021年7月13日,又一批电影历经千辛万苦获得了公映许可证。本次

In [ ]:
 df_curr_releases

In [251]:
df_matched.shape[0]

53

In [ ]:
df_view[df_view['上映日期'] != '上映中']

In [302]:
P1 = ''
P1 +='{Y}年{M}月{D}日,'.format(Y=dt_curr_release.year, M=dt_curr_release.month, D=dt_curr_release.day)
P1 += '又一批电影历经千辛万苦获得了公映许可证。'

P1 += '本次获领证的影片共计{}部，'.format(df_curr_releases.shape[0])
P1 += '距离备案到获取公映证平均超过{}年。'.format(round(df_view['备案至今'].apply(lambda x: float(x.lstrip('> ').rstrip(' 年'))).mean(),1))
P1 += '其中，改过名字的有{}部，'.format(df_view.loc[df_view['备案名'] != '',:].shape[0])
P1 += '换过第一出品单位的有{}部，'.format(df_view.loc[df_view['原备案单位'] != '',:].shape[0])
P1 += '修改后才通过备案的有{}部，'.format(df_view.loc[df_view['是否修改'] == '是',:].shape[0])
if df.shape[0] - df_view.shape[0] > 0:
  P1 += '神秘到无迹可寻的有{}部，'.format(df.shape[0] - df_view.shape[0])
P1 += '题材排名前五的分别是{}'.format('、'.join(df_genre_sorted['类型'][:4]))
P1 += '和{}。'.format(df_genre_sorted['类型'][4:5].iloc[0])

P1 += '目前已在影院放映得有{}部：'.format(
    df_view[df_view['上映日期']== '上映中'].shape[0])
if df_view[df_view['上映日期']== '上映中'].shape[0] > 0:
  P1 += '《{}》。'.format(
      '》、《'.join(df_view.loc[df_view['上映日期'] == '上映中', '片名']))

tbr = df_view[df_view['上映日期'] != '上映中'].dropna()
if tbr.shape[0] > 0:
  P1 += '已确认排期的有'
  for i, f in tbr.iterrows():
    P1 += '{}上画的'.format(f['上映日期'])
    P1 += '《{}》'.format(f['片名'])
    P1 += '、'

  P1 = P1.rstrip('、')
  P1 += '，瓜熟蒂落实属不易。'


print(P1) 

2021年10月14日,又一批电影历经千辛万苦获得了公映许可证。本次获领证的影片共计53部，距离备案到获取公映证平均超过2.3年。其中，改过名字的有16部，换过第一出品单位的有14部，修改后才通过备案的有4部，神秘到无迹可寻的有2部，题材排名前五的分别是都市、其它、传奇、革命和农村。目前已在影院放映得有1部：《我和我的父辈》。


In [282]:
tbr

,片名,备案名,梗概,备案至今,是否修改,第一出品单位,原备案单位,编剧,类型,年代,上映日期,主演,预测片名,kw,主要角色,修正类型,src_img,tgt_img,类型数量


##[7.3] P2 怀胎最久的影片是《红船》,从

In [ ]:
df_view[df_view['片名'] == df_view.iloc[0]['片名']]

In [ ]:
df_view

In [ ]:
record_longest

In [253]:
P2 = ''
P2 += '怀胎最久的影片是《{}》,'.format(df_view.iloc[0]['片名'])
P2 += '从备案到获得公映证超过{}年，'.format(df_view.iloc[0]['备案至今'].lstrip('> ').rstrip(' 年'))
if df_view.iloc[0]['备案名'] != '':
  P2 += '名字从《{}》进化成了《{}》，'.format(df_view.iloc[0]['备案名'], df_view.iloc[0]['片名'])

record_longest = df_view[df_view['片名'] == df_view.iloc[0]['片名']]
P2 += '第一出品方历经了{}'.format('、'.join(record_longest['原备案单位']))
P2 += '最终由{}接手。'.format(df_view.iloc[0]['第一出品单位'])
#ls_choices = ['美好', '胜利', '牛逼']
#P2 += '坚持就是{}。'.format(ls_choices[random.randint(0,len(ls_choices)-1)])

print(P2)

怀胎最久的影片是《天之坑》,从备案到获得公映证超过6.8年，第一出品方历经了广西文化产业投资集团有限公司、最终由云南青派文化传媒有限公司接手。


In [ ]:
df_view

## [7.4] P3 同样坚守的还有...

In [254]:
ls_name_3plus_year = df_view.loc[df_view['备案至今'].apply(
    lambda x: float(x.lstrip('> ').rstrip(' 年')))>3, '片名'].tolist()[1:]
ls_name_3plus_year = ['《'+ name + '》' for name in ls_name_3plus_year]
P3 = ""
P3 += '坚守了3年以上的影片还有{}。'.format('、'.join(ls_name_3plus_year))

print(P3)

坚守了3年以上的影片还有《我的初恋十八岁》、《胶东乳娘》、《江湖狂想曲》、《第一人称》、《山河披锦绣》、《纸骑兵》、《龙门相》、《走出炭子冲》、《异兽》、《浴血大别山》、《遇见你之后》。


## [7.5] P4 上市公司相关

In [255]:
df_curr_releases['第一出品单位'].unique()

array(['国家京剧院', '贵州龙仁影视文化产业有限公司', '四川九王子影视文化传媒有限公司', '庆阳汇恒盛视影视文化传媒有限公司',
       '西安千光影企业文化传播有限公司', '云南雪域光线影业有限公司', '北京边边角角影视文化有限公司', '海南果然传媒有限公司',
       '西藏九尾狐文化传媒有限公司', '厦门风云科技股份有限公司', '甘肃丝路花雨影视文化传媒有限公司',
       '云南青派文化传媒有限公司', '上海咏而归影业有限公司', '北京欢喜首映文化有限公司', '重庆三只喜鹊文化传媒有限公司',
       '陕西润霖影视传媒有限公司', '苏州花儿红电影制作有限公司', '北京七娱世纪文化传媒有限公司', '喜悦之道传媒有限公司',
       '江西电影制片厂有限责任公司', '湖南潇影第二影业有限公司', '湖北电影制片有限责任公司', '宁波影视艺术有限责任公司',
       '中国电影股份有限公司', '佛山市沙漠之狐影视传媒有限公司', '纪录工厂（海南）传媒有限公司',
       '西安盛典长安院线有限责任公司', '陕西影修源影业有限公司', '福建寒武纪影视文化发展有限公司',
       '宿州金盾文化传媒有限责任公司', '威海夏日盛火影视文化有限公司', '北京东方飞云国际影视股份有限公司',
       '北京瀚亿德文化科技有限公司', '电影频道节目中心', '北方联合影视集团有限公司', '北京孔氏兄弟影业有限公司',
       '甘肃鼎级华彩影业有限公司', '天山电影制片厂', '深圳市野火映像文化传播有限公司', '浙江蔓越莓文化传媒有限公司',
       '上海影界文化传播有限公司', '上海肩上鸟影业有限公司', '庆阳黑格儿影视有限公司', '西安乐和影视文化传媒有限公司',
       '河南粉丝文化传媒有限公司', '河北广电盛腾传媒有限责任公司', '浙江自由元素影业有限公司'], dtype=object)

In [310]:
# 本批次中与上市影视公司关联的项目有
info_public_film_co = pd.read_csv('/content/drive/MyDrive/Github/Article/reference/info_public_film_co.csv', index_col=0, encoding='utf-8-sig')
ls = info_public_film_co['公司简称'].apply(eval).sum()
pat_public = '|'.join(ls)
pat_public += '|阿里|腾讯|爱奇艺|英皇|寰亚|银都|美亚|大盛|儒意|灿星|横店|博纳|麦颂'
df_focus = df_view.loc[df_view['第一出品单位'].str.contains(pat_public, regex=True)]
df_focus = df_focus[df_focus['片名'] != '顺子加油']
df_focus

,片名,备案名,梗概,备案至今,是否修改,第一出品单位,原备案单位,编剧,类型,年代,上映日期,主演,预测片名,kw,主要角色,修正类型,src_img,tgt_img,类型数量_x,类型数量_y,类型数量
19,我和我的父辈,,电影《我和我的父辈》由多位导演共同创作，讲述不同历史时期、各个代表性阶层群体不忘初心、砥砺前...,> 0.8 年,,中国电影股份有限公司,,待定,其它,当代,上映中,"[吴京, 章子怡, 徐峥]",我和我的父辈,"[民族, 讲述, 历程, 历史, 时期, 代表性, 阶层, 导演, 精神, 初心]",[],其它,None,/content/drive/MyDrive/Github/Article/img/Chin...,7,7,7
28,峰爆,无限深度,在一场突如其来的灾难面前，老一代铁道兵和新一代铁建工程师齐心合力，拯救身陷险境的孩子们和受灾...,> 1.8 年,,中国电影股份有限公司,,李骏、沙颂,其它,当代,NaN,NaN,铁道兵,"[孩子, 身陷, 拯救, 城市, 受灾, 险境, 故事, 工程师]",[],其它,None,/content/drive/MyDrive/Github/Article/img/Chin...,7,7,7
34,群英会·借东风（京剧）,,三国时，孙、曹对峙于赤壁，刘派诸葛亮到吴营出谋划策。东吴缺箭，诸葛亮施计草船借箭。周用苦肉计...,> 2.1 年,,中国电影股份有限公司,,国家京剧院,传奇,古代,NaN,NaN,诸葛亮,"[战船, 草船借箭, 缺箭, 施计, 责打, 导致, 火烧, 赤壁, 对峙, 周用]","[曹操, 庞统, 刘派, 黄盖, 曹军, 诸葛亮]",传奇,None,/content/drive/MyDrive/Github/Article/img/Chin...,6,6,6
36,盗圣一枝梅,,绝世藏宝图重现人间后，东厂与锦衣卫开始明争暗斗，一枝梅为寻得藏宝图救济百姓，被东厂和锦衣卫双...,> 1.7 年,,云南雪域光线影业有限公司,,单坤、黄瀚,传奇,古代,NaN,NaN,绝世藏宝图,"[东厂, 锦衣卫, 寻得, 夹击, 重现, 人间, 天机, 冲出重围, 带领, 百姓]",[],传奇,None,/content/drive/MyDrive/Github/Article/img/Chin...,6,6,6


In [311]:
idx_to_keep = [19,28]
df_focus_narrowed = df_focus[df_focus.index.isin(idx_to_keep)]

In [312]:
df_focus_narrowed

,片名,备案名,梗概,备案至今,是否修改,第一出品单位,原备案单位,编剧,类型,年代,上映日期,主演,预测片名,kw,主要角色,修正类型,src_img,tgt_img,类型数量_x,类型数量_y,类型数量
19,我和我的父辈,,电影《我和我的父辈》由多位导演共同创作，讲述不同历史时期、各个代表性阶层群体不忘初心、砥砺前...,> 0.8 年,,中国电影股份有限公司,,待定,其它,当代,上映中,"[吴京, 章子怡, 徐峥]",我和我的父辈,"[民族, 讲述, 历程, 历史, 时期, 代表性, 阶层, 导演, 精神, 初心]",[],其它,None,/content/drive/MyDrive/Github/Article/img/Chin...,7,7,7
28,峰爆,无限深度,在一场突如其来的灾难面前，老一代铁道兵和新一代铁建工程师齐心合力，拯救身陷险境的孩子们和受灾...,> 1.8 年,,中国电影股份有限公司,,李骏、沙颂,其它,当代,NaN,NaN,铁道兵,"[孩子, 身陷, 拯救, 城市, 受灾, 险境, 故事, 工程师]",[],其它,None,/content/drive/MyDrive/Github/Article/img/Chin...,7,7,7


In [315]:
P4 = ''
P4 += '本批次中，大数据分析识别出{}部上市影视公司关联项目，'.format(df_focus.shape[0])
P4 += '占比{}%。'.format(round( (df_focus.shape[0]/(df_view.shape[0]-1)*100), 2 ) )
P4 += '结合题材与出品方实力，慷田AI聚焦关注《{}》'.format(
    '》、《'.join(df_focus_narrowed['片名']))
P4 += '，题材类型包括了传记与救灾。'.format(
    '、'.join(df_focus_narrowed.类型.unique()))

print(P4)

本批次中，大数据分析识别出4部上市影视公司关联项目，占比7.69%。结合题材与出品方实力，慷田AI聚焦关注《我和我的父辈》、《峰爆》，题材类型包括了传记与救灾。


##[7.6] P5 按影片展示词云图

In [294]:
df_label2image

,label,fn
0,"[古代, 传奇]",None
1,"[古代, 传记]",vintage-5323524_1920
2,"[古代, 其它]",lampion-1509664_1920
3,"[古代, 军旅]",None
4,"[古代, 农村]",None
...,...,...
60,科幻,jupiter-55000.png
61,都市,cityurban.png
62,青少,art-2687649_1280.png
63,革命,muscle-1085672_1280.png


In [295]:
%%capture
df_view['src_img'] = None
df_view['tgt_img'] = None

In [ ]:
genre_count = df_view.groupby('修正类型')['片名'].count(
    ).rename('类型数量').reset_index()
df_view = df_view.merge(genre_count, on='修正类型', how='left')
df_view = df_view.sort_values(['类型数量','上映日期'], ascending=[False, True])
df_view

In [316]:
P5 = ''
P5 = '\n看图快速了解本期影片。'

print(P5)


看图快速了解本期影片。


In [299]:
idx_actor = df_view['主演'].dropna().index

for i in df_view.drop_duplicates('片名').index:
  ls = [df_view.loc[i, '片名']] * 20
  ls += [df_view.loc[i, '预测片名']] *6
  ls += df_view.loc[i, '主要角色']*3
  if i in idx_actor:
    ls += df_view.loc[i, '主演']*3
  ls += [df_view.loc[i, '修正类型']] *3
  ls += [df_view.loc[i, '年代']]*2
  ls += df_view.loc[i, 'kw']
  ls += [df_view.loc[i, '第一出品单位']]
  img_fn = df_view.loc[i, 'src_img']
  if not img_fn:
    img_fn = df_label2image.loc[
        df_label2image['label'] == df_view.loc[i, '修正类型'],
        'fn'
    ].iloc[0]
  txt = '《{}》'.format(df_view.loc[i, '片名'])
  if not pd.isna(df_view.loc[i, '上映日期']):
    if df_view.loc[i, '上映日期'] == '上映中':
      txt += "({})".format(df_view.loc[i, '上映日期'])
    else:
      txt += "({}上映)".format(df_view.loc[i, '上映日期'])
  txt += '，备案至今{}'.format(df_view.loc[i, '备案至今'])

  if df_view.loc[i, '备案名'] != '':
    txt += '\n原备案片名：《{}》'.format(df_view.loc[i, '备案名'])

  txt += '\n编剧：{}'.format(df_view.loc[i, '编剧'])
  #txt += '\n类型：{}'.format(df_view.loc[i, '类型'])

  txt += '\n第一出品单位：{}'.format(df_view.loc[i, '第一出品单位'])
  if df_view.loc[i, '原备案单位'] != '':
    txt +=  '\n原备案单位：{}'.format(df_view.loc[i, '原备案单位'])
  
  print(txt)
  #txt += '{}\n'.format(df_view.loc[i, '梗概'])
  fp_img = path_img + '/genre_icon/{}'.format(img_fn)
  fp_mask = path_img + '/genre_icon/{}'.format('mask_' + img_fn)
  fp_img = path_img + '/genre_icon/{}'.format('psychedelic-1084082_960_720.jpg')
  fp_generated_img = utils.generate_word_image(ls, 
                                               fp_img, 
                                               fp_mask,
                                               fp_prefix='ChinaFilm_Release_Overview',
                                               fp_suffix=str_date_curr_release,
                                               img_width=400,
                                               )
  #display(Img(fp_generated_img, width=400))
  df_view.loc[i, 'tgt_img'] = fp_generated_img

Output hidden; open in https://colab.research.google.com to view.

In [265]:
P5 = ''
P5 = '看图了解这期推出的电影都讲了什么。'

print(P5)

看图了解这期推出的电影都讲了什么。


## [7.7] 指向电影官网

In [266]:
#####
E1 = '\n\n'
E1 += ''
E1 += '慷田AI结合自主调研及多方大数据比对，通过分析、建模，提炼关键信息。'
E1 += '公映许可证公示信息来自中国国家电影局 China Film Administration, 官方网址 '
E1 += ' http://www.chinafilm.gov.cn/chinafilm 。\n'

print(E1)



慷田AI结合自主调研及多方大数据比对，通过分析、建模，提炼关键信息。公映许可证公示信息来自中国国家电影局 China Film Administration, 官方网址  http://www.chinafilm.gov.cn/chinafilm 。



In [268]:
 df_view['片名'].to_numpy()[0:5]

array(['天之坑', '我的初恋十八岁', '第一人称', '遇见你之后', '邓小平小道'], dtype=object)

## [8.9] E2\E3, Pointer Text

In [267]:
E2 = '点击阅读原文查看本期慷田AI影片信息详表。'
E3 = '点击阅读原文查看本期慷田AI影片概览分析。'

## [7.8] S0 生成摘要

In [320]:
S0 = '{}月新片{}部，'.format(dt_curr_release.month, df_curr_releases.shape[0])
S0 += '平均孕期超{}年，'.format(round(df_view['备案至今'].apply(lambda x: float(x.lstrip('> ').rstrip(' 年'))).mean(),1))
S0 += '改名的{}部，'.format(df_view.loc[df_view['备案名'] != '',:].shape[0])
S0 += '换主的{}部，'.format(df_view.loc[df_view['原备案单位'] != '',:].shape[0])
S0 += '题材集中在{}'.format('、'.join(df_genre_sorted['类型'][:4]))
S0 += '和{}。'.format(df_genre_sorted['类型'][4:5].iloc[0])
S0 += '慷田AI聚焦'
focus = ['《'+ name + '》' for name in df_focus_narrowed['片名'].to_numpy()]
S0 += '{}'.format('、'.join(focus))
S0 += '…'

print(S0)

10月新片53部，平均孕期超2.3年，改名的16部，换主的14部，题材集中在都市、其它、传奇、革命和农村。慷田AI聚焦《我和我的父辈》、《峰爆》…


# [5] Output Word Doc

## [5.1] Install Docx Lib

In [275]:
%%capture
!pip install python-docx
!pip install lxml
from docx import Document
from docx.shared import Inches
from docx.oxml.ns import qn
#from docx.text.parargaph import Paragraph

from docx.shared import Pt

## [5.2] Write Overview to docx and download

In [321]:
path_doc = '/content/drive/MyDrive/Github/Article/docx'
path_img = '/content/drive/MyDrive/Github/Article/img'

doc = Document()

# Set document Font 
doc.styles['Normal'].font.name = '微软雅黑'
r = doc.styles['Normal']._element
r.rPr.rFonts.set(qn('w:eastAsia'), '微软雅黑')

# Set Title
doc.core_properties.title = 'ChinaFilm_NewReleases_Overview_' + str_date_curr_release

doc.add_heading(T1, 0)

p = doc.add_paragraph(S0, style='Intense Quote')

doc.add_paragraph(P1)

doc.add_picture(path_img + '/posters/' + '我和我的父辈.jpg', width =Inches(6))

doc.add_paragraph(P2)

doc.add_paragraph(P3)

doc.add_picture(path_img + '/posters/' + '峰爆.jpg', width =Inches(6))

doc.add_paragraph(P4)

doc.add_paragraph(P5)
for i in df_view.drop_duplicates('片名').index:
  txt = '\n《{}》'.format(df_view.loc[i, '片名'])
  if not pd.isna(df_view.loc[i, '上映日期']):
    if df_view.loc[i, '上映日期'] == '上映中':
      txt += "({})".format(df_view.loc[i, '上映日期'])
    else:
      txt += "({}上映)".format(df_view.loc[i, '上映日期'])
  txt += '，备案至今{}'.format(df_view.loc[i, '备案至今'])
  if df_view.loc[i, '备案名'] != '':
    txt += '\n原备案片名：《{}》'.format(df_view.loc[i, '备案名'])

  txt += '\n编剧：{}'.format(df_view.loc[i, '编剧'])
  #txt += '\n类型：{}'.format(df_view.loc[i, '类型'])

  txt += '\n第一出品单位：{}'.format(df_view.loc[i, '第一出品单位'])
  if df_view.loc[i, '原备案单位'] != '':
    txt +=  '\n原备案单位：{}'.format(df_view.loc[i, '原备案单位'])

  fp_img = df_view.loc[i, 'tgt_img']
  doc.add_paragraph(txt)
  doc.add_picture(fp_img, width =Inches(4))


doc.add_paragraph(E1)
doc.add_paragraph(E2)

doc.save(path_doc + '/ChinaFilm_NewReleases_Overview_' + str_date_curr_release +'.docx')

In [322]:
# Download Document
from google.colab import files

files.download(
    path_doc + '/ChinaFilm_NewReleases_Overview_' + str_date_curr_release +'.docx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## [5.3] Write Table to docx and download

In [323]:
print(T1)

~10月新片的前世今生~2021


In [325]:
path_doc = '/content/drive/MyDrive/Github/Article/docx'

doc = Document()
doc.core_properties.title = 'ChinaFilm_NewReleases_Table_' + str_date_curr_release

doc.add_heading('详表：' + T1, 0)

p = doc.add_paragraph(S0, style='Intense Quote')

table = doc.add_table(rows=1, cols=1, style='Light List Accent 4')
i = 1
for i, row in df_view.iterrows():
  row_cells = table.add_row().cells
  row_cells[0].text = '[' + str(i) + ']' + '《'.format(i) + row['片名'] + '》'
  if row['备案名'] != '':
    row_cells[0].text += ' (原备案名《{}》)'.format(row['备案名'])

  row_cells = table.add_row().cells
  row_cells[0].text = '编剧: ' + row['编剧'] 
  
  row_cells = table.add_row().cells
  row_cells[0].text = '距离备案 ' + row['备案至今']
  if row['是否修改'] == '是':
    row_cells[0].text += ', 修改后通过备案'

  row_cells = table.add_row().cells
  row_cells[0].text = '第一出品单位: ' + row['第一出品单位']
  if row['原备案单位'] != '': 
    row_cells = table.add_row().cells
    row_cells[0].text = '原备案单位：{}'.format(row['原备案单位'])
  
  row_cells = table.add_row().cells
  row_cells[0].text = '类型: ' + row['修正类型'] 

  idx_actor = df_view['主演'].dropna().index
  if i in idx_actor:
    row_cells = table.add_row().cells
    row_cells[0].text = '主演: ' + '、'.join(row['主演']) 

  row_cells = table.add_row().cells
  row_cells[0].text = row['梗概'].lstrip('\w+')

  row_cells = table.add_row().cells
  row_cells[0].text = ''
  i+=1

doc.add_paragraph(E1)
doc.add_paragraph(E3)

doc.save(path_doc + '/ChinaFilm_NewReleases_Table_' +
         str_date_curr_release +'.docx')

In [326]:
files.download(path_doc + '/ChinaFilm_NewReleases_Table_' +
         str_date_curr_release +'.docx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>